In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import TSMixerModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)  # Set the default figure size
torch.set_float32_matmul_precision('medium')  # Set precision for matrix multiplication

In [11]:
# Define target columns for time series forecasting
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

# Load the cleaned ground station data
df = pd.read_csv("../data/ground_station_clean.csv")

In [12]:
# Configure encoders for time features and data transformation
encoders = {
    "cyclic":{  # Cyclic features for time components
        "past":["month","dayofyear","day","hour","minute"],
        "future":["month","dayofyear","day","hour","minute"],
    },
    "transformer": Scaler()  # Data scaling transformer
}

In [13]:
# Custom callback class that combines PyTorch Lightning pruning with a standard callback
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create a directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)  # Create if it doesn't exist

In [14]:
def objective(trial):
    """
    Objective function for Optuna optimization that trains and evaluates a model with given hyperparameters.
    """
    # Define hyperparameters to be optimized
    input_chunk_length = trial.suggest_int("input_chunk_length",1, 72)
    hidden_size = trial.suggest_categorical('hidden_size', [32, 64, 128])
    ff_size = trial.suggest_int('ff_size', hidden_size, hidden_size * 4) # Pode ser dependente
    num_blocks = trial.suggest_int('num_blocks', 1, 4)
    dropout = trial.suggest_float('dropout', 0.0, 0.3, step=0.05)
    activation = trial.suggest_categorical('activation', ['ReLU', 'GELU'])
    norm_type = trial.suggest_categorical('norm_type', ['LayerNormNoBias', 'LayerNorm', 'TimeBatchNorm2d'])
    normalize_before = trial.suggest_categorical('normalize_before', [True, False])
    batch_size = trial.suggest_int("batch_size", 16, 64)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)

    prunner = PatchedPruningCallback(trial, monitor="val_loss")
    # Configure early stopping
    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=10, verbose=True)
    pl_trainer_kwargs = {
            "accelerator": "auto",
            "callbacks": [early_stopper, prunner],
    }

    # Split data into training and validation sets (80/20)
    n = int(len(df)*0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]

    # Convert dataframes to TimeSeries objects
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    # Print trial information
    print(f"\nStarting Trial {trial.number}")
    print(f"Hyperparameters: {trial.params}")
    print("\nTraining the model...")
    print(f"Train set: {len(train_fold)} samples")  # Use len() for TimeSeries objects
    print(f"Validation set: {len(val_fold)} samples")  # Use len() for TimeSeries objects

    # Scale the data using only training data statistics
    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    # Define working directory and model name for consistency
    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/"  # Ensure this path exists and is writable
    _model_name = "model_optuna_temp"  # Temporary model name for each trial

    # Create the working directory if it doesn't exist
    os.makedirs(_work_dir, exist_ok=True)

    # Configure the RNN model with trial hyperparameters
    model = TSMixerModel(
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        output_chunk_length=1,
        hidden_size=hidden_size,
        ff_size=ff_size,
        n_epochs=30,
        batch_size=batch_size,
        norm_type=norm_type,
        normalize_before=normalize_before,
        num_blocks=num_blocks,
        activation=activation,
        dropout=dropout,
        add_encoders=encoders,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=torch.nn.HuberLoss(),
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode":"min", "factor":0.5, "patience":4, "min_lr":1e-6},
        save_checkpoints=True,
        show_warnings=True,
        force_reset=True, # Importante para que cada iteração treine do zero com este model_name
        optimizer_kwargs={"lr": lr, "weight_decay": 1e-5},
    )
    # Train the model
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        dataloader_kwargs={"num_workers": 11}
    )

    # Try to load the best model from checkpoint
    try:
        loaded_model = TSMixerModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Model loaded from checkpoint for trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint not found for {_model_name} in {_work_dir}. Using in-memory trained model.")
        loaded_model = model  # Use the trained model if loading fails (fallback)

    # Generate historical forecasts to evaluate model performance
    forecasts = loaded_model.historical_forecasts(
        train_scaled,  # Use training data
        start=0.8,  # Start forecasting at 80% of the training data
        forecast_horizon=1,  # Forecast one step ahead
        stride=1,  # Generate a forecast at each possible time point
        retrain=False,  # Don't retrain the model for each forecast
    )

    # Inverse transform forecasts and actual values back to an original scale
    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(train_scaled).split_after(0.8)[1]  # Get the actual values for comparison

    # Calculate performance metrics for each target variable
    metrics = {}
    print("Starting time series consistency verification...")
    try:
        for target in target_columns:
            # Calculate multiple error metrics for comprehensive evaluation
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),  # Mean Squared Error
                'RMSE': rmse(s[target], forecasts_t[target]),  # Root Mean Squared Error
                'MAE': mae(s[target], forecasts_t[target]),  # Mean Absolute Error
                'R2': r2_score(s[target], forecasts_t[target]),  # R-squared score
                'SMAPE': smape(s[target], forecasts_t[target]),  # Symmetric Mean Absolute Percentage Error
            }
        # Create a DataFrame for better visualization of metrics
        metrics_df = pd.DataFrame(metrics).T
        print("\nPerformance metrics:")
        print(metrics_df)
    except Exception as e:
        print(e)

    # Calculate overall SMAPE across all variables (optimization target)
    overall_smape_val = smape(s, forecasts_t)
    print(f"The SMAPE for this fold was {overall_smape_val}")

    # Store trial results in a dictionary
    trial_dict = {
            "trial_number": trial.number,
            "input_chunk_length": input_chunk_length,
            "num_blocks": num_blocks,
            "hidden_size": hidden_size,
            "ff_size": ff_size,
            "norm_type": norm_type,
            "normalize_before": normalize_before,
            "activation": activation,
            "dropout": dropout,
            "batch_size": batch_size,
            "lr": lr,
            "overall_smape_val": overall_smape_val,
            "metrics": metrics_df.to_dict(),
        }    # Save trial results to a JSON file
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results for trial {trial.number} saved to {json_path}")

    # Return the optimization metric (SMAPE), or infinity if NaN (to be minimized)
    return overall_smape_val if not np.isnan(overall_smape_val) else float("inf")

In [15]:
# Callback function to print trial progress information
def print_callback(study, trial):
    """Print current trial results and best results so far."""
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [ ]:
# Create and run the optimization study
study = optuna.create_study(direction="minimize")  # We want to minimize SMAPE
study.optimize(objective, n_trials=50, callbacks=[print_callback])

[I 2025-05-19 22:23:09,426] A new study created in memory with name: no-name-082491bf-f3f0-4c77-9301-772a5d252411



Starting Trial 0
Hyperparameters: {'input_chunk_length': 22, 'hidden_size': 128, 'ff_size': 506, 'num_blocks': 1, 'dropout': 0.25, 'activation': 'GELU', 'norm_type': 'LayerNorm', 'normalize_before': False, 'batch_size': 45, 'lr': 2.860274958073652e-05}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type             | Params | Mode 
-------------------------------------------------------------------
0 | criterion             | HuberLoss        | 0      | train
1 | train_criterion       | HuberLoss        | 0      | train
2 | val_criterion         | HuberLoss        | 0      | train
3 | train_metrics         | MetricCollection | 0      | train
4 | val_metrics           | MetricCollection | 0      | train
5 | fc_hist               | Linear           | 23     | train
6 | feature_mixing_hist   | _FeatureMixing   | 83.5 K | train
7 | feature_mixing_future | _FeatureMixing   | 72.1 K | train
8 | conditional_mixer     | ModuleList       | 228 K  | train
9 | fc_out                | Linear           | 1.0 K  | train
-------------------------------------------------------------------
385 K     Trainable par

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Starting time series consistency verification...

Performance metrics:
                                    MSE        RMSE         MAE         R2  \
Temperature                    1.384115    1.176484    0.887705   0.593427   
Precipitation_accumulated     82.614444    9.089249    6.862650 -86.041041   
Humidity                      12.809706    3.579065    2.823070   0.803014   
Wind_Speed_kmh                 0.167527    0.409301    0.229433   0.507720   
Soil_Moisture              84068.380859  289.945479  146.303534   0.886389   
Soil_Temperature               1.188450    1.090160    0.814463   0.788238   
Wind_Dir_Sin                   0.090504    0.300839    0.193692   0.436531   
Wind_Dir_Cos                   0.090954    0.301586    0.172626   0.683189   

                                SMAPE  
Temperature                 14.846278  
Precipitation_accumulated    1.486381  
Humidity                     3.347929  
Wind_Speed_kmh             168.278765  
Soil_Moisture             

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.023


In [8]:
# Print the best results from the optimization
print(f"Best Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best Value (Minimum): 43.56352294806756
Best Parameters: {'input_chunk_length': 61, 'hidden_size': 128, 'ff_size': 153, 'num_blocks': 3, 'dropout': 0.15000000000000002, 'activation': 'GELU', 'norm_type': 'TimeBatchNorm2d', 'normalize_before': False, 'batch_size': 22, 'lr': 0.0004332871234178122}


In [9]:
# Save the best trial results to a separate file
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved to {json_path}")


Best results saved to optuna_iteration_metrics/best_trial.json
